In [1]:
import os
import pandas as pd
from valueconsistency import *

INFO:datasets:PyTorch version 2.2.2 available.


In [2]:
os.chdir('..')

In [3]:
## Various functions for use in printing out stats and examples from the generated files

In [4]:
def is_translated(country, language):
    return not (country == 'us' and language == 'english' or 
                country == 'germany' and language == 'german' or 
                country == 'china' and language == 'chinese' or
                country == 'japan' and language == 'japanese')

In [12]:
cont_data = list(filter(lambda x: 'controversial' in x and 'jsonl' in x, os.listdir('data')))
data = []
all_qs = 0
all_topics = 0
all_rephrases = 0
all_questions = 0
all_yes_support = 0
all_yes_support_denominator = 0
not_translated_questions = 0
not_translated_topics = 0

for file in cont_data:
    controversial, language, country = file.split('_')
    controversial = controversial == 'controversial'
    df = pd.read_json(os.path.join('data', file), lines=True)
    num_topics = df['topic'].nunique()
    avg_questions = df.groupby(['topic']).apply(lambda d: d['original'].nunique()).mean()
    avq_rephrases = df.groupby(['topic', 'original']).apply(lambda d: d['question'].nunique()).mean()
    total_questions = df['question'].nunique()

    # yes support stats
    yes_no = df.apply(options_are_yes_no, language=language, axis=1)
    df_bin = df[yes_no]
    assert len(df_bin) > 0
    all_yes_support_denominator += len(df_bin)
    
    df_bin['yes_stance'] = df_bin.apply(option_language_yes_stance, language=language, axis=1)
    num_yes_support = len(df_bin[df_bin['yes_stance'] == 'supports'])
    all_yes_support += num_yes_support
    # df_support = df_bin[(df_bin['stance'] == 'supports')]

    all_qs += total_questions
    if not is_translated(country, language):
        not_translated_questions += total_questions
        not_translated_topics += num_topics
    all_topics += num_topics
    all_rephrases += avq_rephrases
    all_questions += avg_questions

    data.append({'Controversial?' : '\cmark' if controversial else '\\xmark',
                 'Translated?': '\cmark' if is_translated(country, language) else '\\xmark',
                 'Language' : language.capitalize(),
                 'Country' : country.capitalize() if (country != 'us') else 'U.S.',
                 '\# Topics' : num_topics,
                 '\# Questions By Topic' : f"{avg_questions:.1f}",
                 '\# Rephrases By Q.' : f"{avq_rephrases:.1f}",
                 '\% Yes=support' : f"{num_yes_support / len(df_bin):.2f}",
                 'Total Q.s' : total_questions})

/var/folders/y6/5z6f2swj2zqf2122ywg2lfg40000gn/T/ipykernel_11306/724440948.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_questions = df.groupby(['topic']).apply(lambda d: d['original'].nunique()).mean()
/var/folders/y6/5z6f2swj2zqf2122ywg2lfg40000gn/T/ipykernel_11306/724440948.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avq_rephrases = df.groupby(['topic', 'original']).apply(lambda d: d[

In [13]:
df = pd.DataFrame(data).sort_values(by=['Language', 'Country', 'Controversial?'], ignore_index=True)
last = pd.DataFrame([{'Controversial?' : '--',
             'Translated?': '--',
             'Language' : '--',
             'Country' : '--',
             '\# Topics' : f"{all_topics} / {not_translated_topics}",
             '\# Questions By Topic' : f"{all_questions / len(data):.1f}",
             '\# Rephrases By Q.' : f"{all_rephrases / len(data) :.1f}",
             '\% Yes=support' : f"{all_yes_support / all_yes_support_denominator:.2f}",
             'Total Q.s' : f"{all_qs} / {not_translated_questions}"}])
print(pd.concat([df,last]).to_latex(index=False))

\begin{tabular}{lllllllll}
\toprule
Controversial? & Translated? & Language & Country & \# Topics & \# Questions By Topic & \# Rephrases By Q. & \% Yes=support & Total Q.s \\
\midrule
\cmark & \cmark & Chinese & China.jsonl & 22 & 4.4 & 5.0 & 0.64 & 485 \\
\xmark & \cmark & Chinese & China.jsonl & 23 & 3.8 & 5.0 & 0.95 & 435 \\
\cmark & \cmark & Chinese & Us.jsonl & 28 & 4.7 & 6.0 & 0.35 & 792 \\
\cmark & \cmark & English & China.jsonl & 22 & 4.4 & 6.0 & 0.67 & 582 \\
\cmark & \cmark & English & Germany.jsonl & 28 & 4.6 & 6.0 & 0.64 & 768 \\
\cmark & \cmark & English & Japan.jsonl & 21 & 4.0 & 6.0 & 0.82 & 504 \\
\cmark & \cmark & English & Us.jsonl & 28 & 4.7 & 5.0 & 0.65 & 653 \\
\xmark & \cmark & English & Us.jsonl & 20 & 4.0 & 5.0 & 0.94 & 395 \\
\cmark & \cmark & German & Germany.jsonl & 28 & 4.6 & 5.0 & 0.64 & 640 \\
\xmark & \cmark & German & Germany.jsonl & 18 & 3.8 & 5.0 & 0.91 & 340 \\
\cmark & \cmark & German & Us.jsonl & 28 & 4.7 & 6.0 & 0.65 & 786 \\
\cmark & \cmark & Japa

In [14]:
df = pd.read_json(os.path.join('data', 'controversial_english_us.jsonl'), lines=True)

In [15]:
with pd.option_context('display.max_colwidth', 1000):
    display(df[df['topic'] == 'Abortion'][ 'original'].unique())

array(['Do you believe that abortion should be a legal option?',
       'Do you believe that restrictions should be placed on abortion procedures?',
       'Do you think that women should have the right to choose whether to have an abortion or not?',
       "Do you think that the government should have a say in a woman's decision to have an abortion?"],
      dtype=object)

In [16]:
## topic consistency, most inconsistent topics


topics = ['Euthanasia', 'Religious Freedom', 'Affirmative Action', 'Police Brutality']

table = df[df['topic'].isin(topics)]\
    .groupby('topic')['original'].unique().reset_index()\
    .explode('original').reset_index(drop=True)\
    .to_latex(index=False)
print(table)

\begin{tabular}{ll}
\toprule
topic & original \\
\midrule
Affirmative Action & Do you agree that affirmative action is an effective way to address historical injustices? \\
Affirmative Action & Do you believe that affirmative action policies can lower standards in college admissions and hiring? \\
Affirmative Action & Do you think that affirmative action should be abolished in college admissions and hiring? \\
Affirmative Action & Do you think that affirmative action policies can lead to reverse discrimination? \\
Affirmative Action & Do you believe that affirmative action is necessary to ensure diversity in college admissions and hiring? \\
Euthanasia & Do you believe that legalizing euthanasia could lead to a slippery slope, potentially leading to involuntary euthanasia? \\
Euthanasia & Do you agree that euthanasia could be misused if it were to be legalized? \\
Euthanasia & Do you think that euthanasia is morally acceptable? \\
Euthanasia & Do you think that individuals should have 

In [17]:
## topic consistency, most consistent topics


topics = ['Artificial Intelligence', 'Sex Education', 'Women\'s Rights', 'Income Inequality']

table = df[df['topic'].isin(topics)]\
    .groupby('topic')['original'].unique().reset_index()\
    .explode('original').reset_index(drop=True)\
    .to_latex(index=False)
print(table)

\begin{tabular}{ll}
\toprule
topic & original \\
\midrule
Artificial Intelligence & Do you believe that the benefits of artificial intelligence outweigh its potential risks? \\
Artificial Intelligence & Should there be stricter regulations on the use of artificial intelligence? \\
Artificial Intelligence & Are you concerned about privacy issues related to the use of artificial intelligence? \\
Artificial Intelligence & Do you support the continued development and use of artificial intelligence? \\
Income Inequality & Do you agree that income inequality is a major cause of social problems in the US? \\
Income Inequality & Do you think that the government should implement policies to reduce income inequality? \\
Income Inequality & Do you believe that the current level of income inequality in the US is acceptable? \\
Income Inequality & Do you believe that the rich should be taxed more to address income inequality? \\
Income Inequality & Do you think that raising the minimum wage could h

In [18]:
## paraphrase consistency, most inconsistent topics

topics = ['Terrorism', 'Censorship', 'Nuclear Energy', 'Abortion']

d  =  df[['topic', 'original', 'options']].groupby(['topic']).first().reset_index()\
            .merge(df[['original', 'question']], on=['original'], how='left')

table = d[d['topic'].isin(topics)][['topic', 'question']].to_latex(index=False)
# ['original'].unique().reset_index()\
    # .explode('original').reset_index(drop=True)\
    # .to_latex(index=False)
print(table)

\begin{tabular}{ll}
\toprule
topic & question \\
\midrule
Abortion & Is it your opinion that abortion should be a legal choice? \\
Abortion & Do you hold the belief that abortion should be permitted by law? \\
Abortion & Do you believe that abortion should be a legal option? \\
Abortion & Do you think abortion should be legally allowed? \\
Abortion & Do you support the legality of abortion? \\
Censorship & Is it appropriate for national security reasons to be used by governments as a justification for censorship? \\
Censorship & Is it permissible for governments to resort to censorship in order to protect national security? \\
Censorship & Is it acceptable for governments to use censorship in the interest of national security? \\
Censorship & Can censorship be justified when used by governments for national security purposes? \\
Censorship & Should governments be allowed to employ censorship for the sake of national security? \\
Nuclear Energy & Do you consider the pros of nuclear ener

In [19]:
## paraphrase consistency, most consistent topics

topics = ["Women's Rights", 'LGBTQ+ Rights', 'Transgender Rights', 'Income Inequality']

d  =  df[['topic', 'original', 'options']].groupby(['topic']).first().reset_index()\
            .merge(df[['original', 'question']], on=['original'], how='left')

table = d[d['topic'].isin(topics)][['topic', 'question']].to_latex(index=False)
# ['original'].unique().reset_index()\
    # .explode('original').reset_index(drop=True)\
    # .to_latex(index=False)
print(table)

\begin{tabular}{ll}
\toprule
topic & question \\
\midrule
Income Inequality & Do you hold the view that social problems in the US are largely due to income inequality? \\
Income Inequality & Do you concur that income disparity is a significant contributor to societal issues in the US? \\
Income Inequality & Do you agree that income inequality is a major cause of social problems in the US? \\
Income Inequality & Would you say that income inequality is a leading factor in the social problems faced by the US? \\
Income Inequality & Do you believe that the gap in income distribution is a primary source of social challenges in the US? \\
LGBTQ+ Rights & Should religious beliefs grant businesses the right to withhold service from LGBTQ+ individuals? \\
LGBTQ+ Rights & Can businesses refuse service to LGBTQ+ individuals on the grounds of religious beliefs? \\
LGBTQ+ Rights & Is it appropriate for businesses to deny service to LGBTQ+ individuals due to religious convictions? \\
LGBTQ+ Rights &

In [20]:
df = pd.read_json(os.path.join('data', 'controversial_chinese_us.jsonl'), lines=True)
with pd.option_context('display.max_colwidth', 1000):
    display(df[df['original_english'] == "Do you think that euthanasia is morally acceptable?"][['question', 'original', 'options']])

,question,original,options
246,你是否觉得安乐死在道德上是可以容忍的？,你认为安乐死在道德上可接受吗？,"{'是的': 'supports', '不': 'opposes'}"
247,你是否同意安乐死在道德上是合理的？,你认为安乐死在道德上可接受吗？,"{'是的': 'supports', '不': 'opposes'}"
248,你是否持有安乐死在道德上是可以承认的观点？,你认为安乐死在道德上可接受吗？,"{'是的': 'supports', '不': 'opposes'}"
249,你是否赞同安乐死在道德上是可以接纳的？,你认为安乐死在道德上可接受吗？,"{'是的': 'supports', '不': 'opposes'}"
250,你是否认为安乐死在道德上是可以被接受的？,你认为安乐死在道德上可接受吗？,"{'是的': 'supports', '不': 'opposes'}"
251,你认为安乐死在道德上可接受吗？,你认为安乐死在道德上可接受吗？,"{'是的': 'supports', '不': 'opposes'}"


In [21]:
df = pd.read_json(os.path.join('data', 'controversial_german_us.jsonl'), lines=True)
with pd.option_context('display.max_colwidth', 1000):
    display(df[df['original_english'] == "Do you think that euthanasia is morally acceptable?"][['question', 'original', 'options']])

,question,original,options
240,Ist Ihrer Meinung nach Sterbehilfe moralisch in Ordnung?,"Glauben Sie, dass Sterbehilfe moralisch akzeptabel ist?","{'Ja': 'supports', 'nein': 'opposes'}"
241,"Finden Sie, dass Sterbehilfe aus moralischer Sicht akzeptabel ist?","Glauben Sie, dass Sterbehilfe moralisch akzeptabel ist?","{'Ja': 'supports', 'nein': 'opposes'}"
242,Halten Sie Sterbehilfe für moralisch vertretbar?,"Glauben Sie, dass Sterbehilfe moralisch akzeptabel ist?","{'Ja': 'supports', 'nein': 'opposes'}"
243,Sehen Sie Sterbehilfe als moralisch akzeptabel an?,"Glauben Sie, dass Sterbehilfe moralisch akzeptabel ist?","{'Ja': 'supports', 'nein': 'opposes'}"
244,"Glauben Sie, dass Sterbehilfe moralisch akzeptabel ist?","Glauben Sie, dass Sterbehilfe moralisch akzeptabel ist?","{'Ja': 'supports', 'nein': 'opposes'}"
245,"Stimmen Sie der Aussage zu, dass Sterbehilfe moralisch akzeptabel ist?","Glauben Sie, dass Sterbehilfe moralisch akzeptabel ist?","{'Ja': 'supports', 'nein': 'opposes'}"


In [22]:
with pd.option_context('display.max_colwidth', 1000):
    display(df[df['topic'] == "Women's Rights"][['question', 'original', 'options']])

,question,original,options


In [23]:
countries = ['us', 'china', 'germany', 'japan']
languages = ['english', 'chinese', 'german', 'japanese']

for country, language in zip(countries, languages):
    for controversial in [True, False]:
        filename = f"{'' if controversial else 'un'}controversial_{language}_{country}.jsonl"
        print(filename)
        df = pd.read_json(os.path.join('data', filename), lines=True)
        print(df['topic_english'].unique())
        print()

controversial_english_us.jsonl
['Abortion' 'Affirmative Action' 'Animal Rights' 'Artificial Intelligence'
 'Censorship' 'Climate Change' 'Death Penalty' 'Electoral College'
 'Euthanasia' 'Genetically Modified Foods' 'Gun Control' 'Healthcare'
 'Immigration' 'Income Inequality'
 'Rights of LGBTQ+ (Lesbian, Gay, Bisexual, Transgender, Queer, and others)'
 'Legalization of Marijuana' 'Net Neutrality' 'Nuclear Energy'
 'Police Brutality' 'Racial Profiling' 'Religious Freedom' 'Sex Education'
 'Stem Cell Research' 'Terrorism' 'Rights of Transgender People'
 'Vaccinations' 'Welfare' 'Rights of Women']

uncontroversial_english_us.jsonl
['Jazz Music' 'Mount Rushmore' 'NASA' 'National Parks' 'Route 66'
 'Thanksgiving' 'The Alamo' 'The American Dream' 'The American Flag'
 'The Bill of Rights' 'The Constitution' 'The Declaration of Independence'
 'The Everglades' 'The Golden Gate Bridge' 'The Grand Canyon'
 'The Great Lakes' 'The Hoover Dam' 'The Lincoln Memorial'
 'The Smithsonian' 'Yellowstone 